Act 8. Programación Python para Big Data

Alumno: Jose Sepulveda

In [1]:
from pyspark.sql import SparkSession 

## Actividad 

El/la alumno/a deberá entregar un archivo .ipynb como actividad con las
siguientes partes:

**Actividad 1** (2,0 ptos)

Transformar en vector los datos numéricos usando
VectorAssembler

**Respuesta:**


In [2]:
# importamos funciones necesarias
from pyspark.ml.feature import VectorAssembler 
from pyspark.sql.functions import col
# creamos sesion
spark = SparkSession.builder.appName('titanic').getOrCreate() 

# cargamos datos
df = spark.read.csv('data/iris.csv', header = True).cache()

# se cambia el tipo de datos de las variables
df = df.select(col('sepal_length').cast('float'), 
               col('sepal_width').cast('float'), 
               col('petal_length').cast('float'), 
               col('petal_width').cast('float'), 
               col('species') )

# seleccion de field numericos
required_features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'] 

# creamos instancia
assembler = VectorAssembler(inputCols=required_features, outputCol='numeric_fields') 

# transmormamos data
df_trans = assembler.transform(df) 

df_trans.show(10)

21/09/23 12:58:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+------------+-----------+------------+-----------+-------+--------------------+
|sepal_length|sepal_width|petal_length|petal_width|species|      numeric_fields|
+------------+-----------+------------+-----------+-------+--------------------+
|         5.1|        3.5|         1.4|        0.2| setosa|[5.09999990463256...|
|         4.9|        3.0|         1.4|        0.2| setosa|[4.90000009536743...|
|         4.7|        3.2|         1.3|        0.2| setosa|[4.69999980926513...|
|         4.6|        3.1|         1.5|        0.2| setosa|[4.59999990463256...|
|         5.0|        3.6|         1.4|        0.2| setosa|[5.0,3.5999999046...|
|         5.4|        3.9|         1.7|        0.4| setosa|[5.40000009536743...|
|         4.6|        3.4|         1.4|        0.3| setosa|[4.59999990463256...|
|         5.0|        3.4|         1.5|        0.2| setosa|[5.0,3.4000000953...|
|         4.4|        2.9|         1.4|        0.2| setosa|[4.40000009536743...|
|         4.9|        3.1|  

**Actividad 2** (2,0 ptos)

Transformar los datos correspondientes a la columna especies
usando StringIndexer

**Respuesta:**

In [3]:
# importamos funciones necesarias
from pyspark.ml.feature import StringIndexer 

# transformamos variable string
df_trans = StringIndexer(inputCol='species', outputCol='type', handleInvalid='keep')\
    .fit(df).transform(df_trans) 

# eliminamos variable species 
df_trans = df_trans.drop('species')
df_trans.show(10)

+------------+-----------+------------+-----------+--------------------+----+
|sepal_length|sepal_width|petal_length|petal_width|      numeric_fields|type|
+------------+-----------+------------+-----------+--------------------+----+
|         5.1|        3.5|         1.4|        0.2|[5.09999990463256...| 0.0|
|         4.9|        3.0|         1.4|        0.2|[4.90000009536743...| 0.0|
|         4.7|        3.2|         1.3|        0.2|[4.69999980926513...| 0.0|
|         4.6|        3.1|         1.5|        0.2|[4.59999990463256...| 0.0|
|         5.0|        3.6|         1.4|        0.2|[5.0,3.5999999046...| 0.0|
|         5.4|        3.9|         1.7|        0.4|[5.40000009536743...| 0.0|
|         4.6|        3.4|         1.4|        0.3|[4.59999990463256...| 0.0|
|         5.0|        3.4|         1.5|        0.2|[5.0,3.4000000953...| 0.0|
|         4.4|        2.9|         1.4|        0.2|[4.40000009536743...| 0.0|
|         4.9|        3.1|         1.5|        0.1|[4.9000000953

**Actividad 3** (2,0 ptos)

Calcular la predicción y la precisión del modelo empleando Decision Tree Classifier

**Respuesta:**

In [4]:
# importamos funciones necesarias
from pyspark.ml.classification import DecisionTreeClassifier 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# creamos train and test df
(df_train, df_test) = df_trans.randomSplit([0.8,0.2],44)

# inicializamos modele
dt = DecisionTreeClassifier(labelCol='type', featuresCol='numeric_fields', maxDepth=5)

# entrenamos model
model = dt.fit(df_train)

# hacemos prediccion
predictions = model.transform(df_test)

# calculamos metrica
evaluator = MulticlassClassificationEvaluator(labelCol='type', 
                                              predictionCol='prediction', 
                                              metricName='weightedPrecision') 

precision = evaluator.evaluate(predictions) 
print('Precision del test = ', precision)

Precision del test =  0.9444444444444444


**Actividad 4** (2,0 ptos)

Calcular la predicción y la precisión del modelo empleando
Gradient-boosted tree classifier

**Respuesta:**

Gradient-boosted tree classifier solo soporta target binario, en este caso el target tiene 3 labels. Se cambia el modelo a utilizar por Naive Bayes

In [5]:
# importamos funciones necesarias
from pyspark.ml.classification import NaiveBayes

# inicializamos modele
nb = NaiveBayes(labelCol="type", featuresCol="numeric_fields", 
                smoothing=1.0, modelType="multinomial")

# entrenamos model
model = nb.fit(df_train)

# hacemos prediccion
predictions = model.transform(df_test)

# calculamos metrica
evaluator = MulticlassClassificationEvaluator(labelCol='type', 
                                              predictionCol='prediction', 
                                              metricName='weightedPrecision') 

precision = evaluator.evaluate(predictions) 
print('Precision del test = ', precision)

Precision del test =  0.9345238095238095


21/09/23 12:58:56 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/09/23 12:58:56 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


**Actividad 5** (2,0 ptos)

Calcular la predicción y la precisión del modelo empleando Random
Forest Classifier

**Respuesta:**

In [6]:
# importamos funciones necesarias
from pyspark.ml.classification import RandomForestClassifier

# inicializamos modele
rf = RandomForestClassifier(labelCol="type", featuresCol="numeric_fields", numTrees=10)

# entrenamos model
model = rf.fit(df_train)

# hacemos prediccion
predictions = model.transform(df_test)

# calculamos metrica
evaluator = MulticlassClassificationEvaluator(labelCol='type', 
                                              predictionCol='prediction', 
                                              metricName='weightedPrecision') 

precision = evaluator.evaluate(predictions) 
print('Precision del test = ', precision)

Precision del test =  1.0
